In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
movie_df = pd.read_csv('./data/movies.csv')
movie_df.keys()

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [3]:
movie_df.describe(
)

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [4]:
movie_df.isnull().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [5]:
movie_df = movie_df[ ['id', 'title', 'genre', 'overview' ] ]#= movie_df['id'], movie_df['title'], movie_df['genres'], movie_df['overview'], movie_df['popularity']

In [6]:
movie_df.head()

,id,title,genre,overview
0,278,The Shawshank Redemption,"Drama,Crime",Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,"Drama,History,War",The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,"Drama,Crime",In the continuing saga of the Corleone crime f...


In [7]:
movie_df['tags'] = movie_df['overview']  + movie_df['genre']
movie_df.drop(columns=['overview', 'genre'], inplace=True)
movie_df

,id,title,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...
...,...,...,...
9995,10196,The Last Airbender,"The story follows the adventures of Aang, a yo..."
9996,331446,Sharknado 3: Oh Hell No!,The sharks take bite out of the East Coast whe...
9997,13995,Captain America,"During World War II, a brave, patriotic Americ..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,A man named Farmer sets out to rescue his kidn...


In [8]:
cv = CountVectorizer(max_features=10000, stop_words='english') # max_features=10000, stop_words='english' , converts text into matrix of token counts

In [9]:
vector = cv.fit_transform(movie_df['tags'].values.astype('U')).toarray() # converts text into matrix of token counts
vector.shape

(10000, 10000)

In [10]:
similarity = cosine_similarity(vector)
similarity.shape

(10000, 10000)

In [11]:
similarity[2].argsort()[:-11:-1] # top 10 similar movies
for i in range(0, 5):
    print(movie_df['title'][i])
    print(list(movie_df['title'][similarity[i].argsort()[-2:-12:-1]]))
    print()

The Shawshank Redemption
['Brubaker', 'The Woodsman', 'A Prophet', 'Gotti', 'Cool Hand Luke', 'Life', 'Synecdoche, New York', 'American Gigolo', 'Dark Passage', 'The Unforgivable']

Dilwale Dulhania Le Jayenge
['A Passage to India', 'The Manual of Love', 'The Cameraman', 'Smart People', 'A California Christmas', 'The Graduate', 'The Broken Circle Breakdown', 'The Nanny Diaries', 'The Awful Truth', 'Just Married']

The Godfather
['The Godfather: Part II', 'Blood Ties', 'Joker', 'Bomb City', 'Gotti', 'Felon', 'Rope', 'Batman: The Killing Joke', 'The Big Heat', 'The Outsider']

Schindler's List
['Resistance', 'The Counterfeiters', 'The Railway Man', 'Midway', 'Red Tails', 'A Hidden Life', 'Defiance', 'A Man Escaped', 'Black Book', 'The Aftermath']

The Godfather: Part II
['The Godfather', 'The Godfather: Part III', 'Gotti', 'Mulholland Falls', 'Dead Man Down', 'Barton Fink', 'The Wild Goose Lake', 'The Goldfinch', 'The Wizards Return: Alex vs. Alex', 'A Most Violent Year']



In [12]:
distance_godfather =  sorted(list(enumerate(similarity[2])), key=lambda x: x[1], reverse=True)[1:11] # top 10 similar movies
distance_godfather

[(4, 0.48976229911514363),
 (7419, 0.3521803625302496),
 (153, 0.3354968547317302),
 (2624, 0.3234983196103152),
 (9520, 0.3112864031823452),
 (2412, 0.3081578172139684),
 (330, 0.30499714066520933),
 (5010, 0.2995012465378748),
 (779, 0.29606845410646954),
 (7049, 0.29606845410646954)]

In [13]:
# # use clip to encode text embeddings
# from transformers import CLIPProcessor, CLIPModel

# # Load the CLIP model and processor
# model_name = "openai/clip-vit-base-patch32"
# processor = CLIPProcessor.from_pretrained(model_name)
# model = CLIPModel.from_pretrained(model_name)

# # Prepare inputs
# image_path = "path_to_your_image.jpg"
# text = "a photo of a cat"
# inputs = processor(text, images=image_path, return_tensors="pt", padding=True)

# # Get embeddings
# outputs = model(**inputs)
# text_embedding = outputs.text_embeds  # Embedding for the text
# image_embedding = outputs.image_embeds  # Embedding for the image

# # Print embeddings
# print("Text Embedding:", text_embedding)
# print("Image Embedding:", image_embedding)


In [14]:
def recommend(movies):
    index = movie_df[movie_df['title'] == movies].index[0]
    distance_godfather =  sorted(list(enumerate(similarity[index])), key=lambda x: x[1], reverse=True)[1:11] 
    for i in distance_godfather:
        print(movie_df.iloc[i[0]].title)


In [15]:
recommend('The Godfather')

The Godfather: Part II
Blood Ties
Joker
Bomb City
Gotti
Felon
Rope
Batman: The Killing Joke
The Big Heat
The Outsider


In [17]:
pickle.dump(movie_df, open('movie_df.pkl', 'wb'))
pickle.dump(movie_df, open ('similarity.pkl', 'wb'))

In [ ]:
pickle.load(open('movie_df.pkl', 'rb'))